****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('data/world_bank_projects.json', convert_dates=True)
df.head(1)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...


##### Below is the solution the problem 1
1. Find the 10 countries with most projects

In [3]:
#count by country name and sort decending
top10 = df.groupby('countryname').size().sort_values(ascending=False)

#dropping Africa as it is not a country
top10 = top10.drop('Africa')

#printing top ten country with most projects
top10 = top10.reset_index()
top10.index = top10.index + 1
top10.columns = ['countryname', 'value']
top10.head(10)

,countryname,value
1,People's Republic of China,19
2,Republic of Indonesia,19
3,Socialist Republic of Vietnam,17
4,Republic of India,16
5,Republic of Yemen,13
6,Nepal,12
7,People's Republic of Bangladesh,12
8,Kingdom of Morocco,12
9,Republic of Mozambique,11
10,Burkina Faso,9


In [4]:
t = df.mjtheme_namecode #theme variable
t.head() #see what the data looks like

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

In [5]:
t[1] #deeper look

[{'code': '1', 'name': 'Economic management'},
 {'code': '6', 'name': 'Social protection and risk management'}]

In [6]:
#dictionary for counting
codename = {}

for n in t: #forloop all the rows
    for i in n: #forloop all the separate project themes
        if i['code']+','+i['name'] in codename.keys():#if already in dictionary, add 1 to value
            codename[i['code']+','+i['name']] += 1
        else: #else create key with the value of 1
            codename[i['code']+','+i['name']] = 1
            
codename

{'1,': 5,
 '1,Economic management': 33,
 '10,': 14,
 '10,Rural development': 202,
 '11,': 27,
 '11,Environment and natural resources management': 223,
 '2,': 15,
 '2,Public sector governance': 184,
 '3,': 3,
 '3,Rule of law': 12,
 '4,': 16,
 '4,Financial and private sector development': 130,
 '5,': 5,
 '5,Trade and integration': 72,
 '6,': 10,
 '6,Social protection and risk management': 158,
 '7,': 11,
 '7,Social dev/gender/inclusion': 119,
 '8,': 13,
 '8,Human development': 197,
 '9,': 3,
 '9,Urban development': 47}

In [7]:
cn = list(codename.items())#convert to list of tuples
codenamedf = pd.DataFrame(cn)#convert to pandas DataFrame
codenamedf

,0,1
0,"8,Human development",197
1,"11,",27
2,"1,Economic management",33
3,"6,Social protection and risk management",158
4,"5,Trade and integration",72
5,"2,Public sector governance",184
6,"11,Environment and natural resources management",223
7,"7,Social dev/gender/inclusion",119
8,"4,Financial and private sector development",130
9,"6,",10


In [8]:
#separate 'code' from 'name' into a new column
c = pd.concat([codenamedf[0].str.split(',', expand=True), codenamedf[[1]]], axis=1)
c.columns = ['code', 'name', 'value']#rename columns
c = c.set_index(c.code) #set index to code 
c = c.drop('code', axis=1)#for some reason I couldn't get it to drop the column
c.head()

,name,value
code,,
8,Human development,197
11,,27
1,Economic management,33
6,Social protection and risk management,158
5,Trade and integration,72


In [9]:
c = c.replace(r'', np.NaN, regex=True)#replace empty strings with NaN Values
c_null = c[c.name.isnull()]# separating null values
c_null#table with NaN value for 'name'

,name,value
code,,
11,NaN,27
6,NaN,10
8,NaN,13
7,NaN,11
2,NaN,15
4,NaN,16
10,NaN,14
5,NaN,5
1,NaN,5


In [10]:
c_drop = c.dropna()#Dropped values
c_drop #table with no NaN values

,name,value
code,,
8,Human development,197
1,Economic management,33
6,Social protection and risk management,158
5,Trade and integration,72
2,Public sector governance,184
11,Environment and natural resources management,223
7,Social dev/gender/inclusion,119
4,Financial and private sector development,130
10,Rural development,202


In [11]:
mt = c_drop.merge(c_null, how='left', right_index=True, left_index=True) #merged tables
mt = mt.drop('name_y', axis=1) #remove NaN column
mt

,name_x,value_x,value_y
code,,,
8,Human development,197,13
1,Economic management,33,5
6,Social protection and risk management,158,10
5,Trade and integration,72,5
2,Public sector governance,184,15
11,Environment and natural resources management,223,27
7,Social dev/gender/inclusion,119,11
4,Financial and private sector development,130,16
10,Rural development,202,14


#### Below is solution to question 2
2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [12]:
mt['total'] = mt.value_x + mt.value_y #create a total column with the sum of values
mt = mt.sort_values('total', ascending=False)#sorting wasn't necessary as the order was the same
mt = mt.reset_index() #reset index to range
mt.index = mt.index + 1 #set index to start at 1
mt.head(10) #list only top 10

,code,name_x,value_x,value_y,total
1,11,Environment and natural resources management,223,27,250
2,10,Rural development,202,14,216
3,8,Human development,197,13,210
4,2,Public sector governance,184,15,199
5,6,Social protection and risk management,158,10,168
6,4,Financial and private sector development,130,16,146
7,7,Social dev/gender/inclusion,119,11,130
8,5,Trade and integration,72,5,77
9,9,Urban development,47,3,50
10,1,Economic management,33,5,38


#### below is trying to solve 3
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [13]:
#Creating a dictionary for code/name
code, name = mt.code, mt.name_x
codedict = {code.iloc[row]: name.iloc[row] for row in range(len(mt))} #using iloc as the variable mt isn't zero indexed
codedict

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [14]:
#creating a function to replace
df2 = pd.read_json('data/world_bank_projects.json', convert_dates=True)
mjt = df2.mjtheme_namecode

def fillempty(themes):
    filled = [] 
    for theme in themes:
        if theme['name'] == '': #if 'name' value is blank, 
            theme['name'] = codedict[theme['code']] #replace blank with value of codedict from key
        filled.append(theme)
    return filled

test = mjt[0] #testing to see if it works on one row
print('before:')
print(test)
test = fillempty(test)
print('after:')
print(test)

before:
[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
after:
[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': 'Environment and natural resources management'}]


In [15]:
mjt[12]

[{'code': '4', 'name': ''}]

In [16]:
%%time
for row in range(len(mjt)):
    mjt[row] = fillempty(mjt[row])


C:\Python\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Wall time: 11.9 s


In [17]:
mjt[12]

[{'code': '4', 'name': 'Financial and private sector development'}]

### it is slow and it raises a warning, but it seems to work

In [18]:
df2.loc[12]

_id                                      {'$oid': '52b213b38594d8a2be17c78c'}
approvalfy                                                               2014
board_approval_month                                                  October
boardapprovaldate                                        2013-10-24T00:00:00Z
borrower                                                  GOVERNMENT OF GHANA
closingdate                                              2019-06-30T00:00:00Z
country_namecode                                       Republic of Ghana!$!GH
countrycode                                                                GH
countryname                                                 Republic of Ghana
countryshortname                                                        Ghana
docty                       Project Appraisal Document,Integrated Safeguar...
envassesmentcategorycode                                                    C
grantamt                                                        

In [19]:
#Count again and see if I get the same results

codename = {}

for n in mjt: #using same code from above except new variable mjt
    for i in n: #forloop all the separate project themes
        if i['code']+','+i['name'] in codename.keys():#if already in dictionary, add 1 to value
            codename[i['code']+','+i['name']] += 1
        else: #else create key with the value of 1
            codename[i['code']+','+i['name']] = 1
            
codename

{'1,Economic management': 38,
 '10,Rural development': 216,
 '11,Environment and natural resources management': 250,
 '2,Public sector governance': 199,
 '3,Rule of law': 15,
 '4,Financial and private sector development': 146,
 '5,Trade and integration': 77,
 '6,Social protection and risk management': 168,
 '7,Social dev/gender/inclusion': 130,
 '8,Human development': 210,
 '9,Urban development': 50}

In [20]:
f = pd.DataFrame(list(codename.items())).sort_values(1, ascending=False).reset_index()
f.index = f.index + 1 #making sure index is identical to mt
f

,index,0,1
1,1,"11,Environment and natural resources management",250
2,8,"10,Rural development",216
3,0,"8,Human development",210
4,5,"2,Public sector governance",199
5,3,"6,Social protection and risk management",168
6,7,"4,Financial and private sector development",146
7,6,"7,Social dev/gender/inclusion",130
8,4,"5,Trade and integration",77
9,9,"9,Urban development",50
10,2,"1,Economic management",38


In [21]:
f[1] == mt.total #total is the same

1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
dtype: bool